# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [262]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [205]:
weather_df = pd.read_csv('../Weatherpy/output_data/cities.csv')

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudlness,Wind Speed,Country,Date
0,Esmeraldas,-19.7625,-44.3139,17.00,82,0,1.03,BR,1619419046
1,Lyubinskiy,55.1542,72.6925,16.00,39,75,7.00,RU,1619419046
2,Jamestown,42.0970,-79.2353,-0.56,86,90,4.12,US,1619419047
3,Bengkulu,-3.8004,102.2655,28.64,77,80,3.44,ID,1619418992
4,Ushuaia,-54.8000,-68.3000,6.00,93,90,3.60,AR,1619419034
...,...,...,...,...,...,...,...,...,...
584,Marabu,4.4148,114.0089,30.00,66,40,3.09,MY,1619418964
585,Margate,51.3813,1.3862,6.67,65,35,0.89,GB,1619418996
586,Calabozo,8.9242,-67.4293,27.25,58,95,3.77,VE,1619419254
587,Leh,34.1667,77.5833,6.72,26,0,0.98,IN,1619419254


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [206]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humidity as the weight

locations = weather_df[['Lat' , 'Lng']].astype(float)

humidity = weather_df['Humidity'].astype(float)


In [207]:
# Create a heatmap layer

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating = False, max_intensity = 100,
                                point_radius = 1)
fig.add_layer(heat_layer)


fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [215]:
narrow_cities_df = weather_df.loc[(weather_df['Max Temp'] > 30) & (weather_df['Max Temp']< 40) & 
                                  (weather_df['Wind Speed'] < 10) & (weather_df['Cloudlness'] == 0)].dropna()


narrow_cities_df.head(5)


,City,Lat,Lng,Max Temp,Humidity,Cloudlness,Wind Speed,Country,Date
54,Sakrand,26.1384,68.2736,39.0,27,0,4.12,PK,1619419064
82,Karratha,-20.7377,116.8463,30.6,24,0,6.60,AU,1619419072
100,Rāmpur,28.8167,79.0333,39.5,4,0,4.65,IN,1619419078
190,Nan,18.8333,100.6667,31.0,66,0,1.03,TH,1619418896
268,Ahvāz,31.3203,48.6693,32.0,35,0,2.06,IR,1619419137


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create dataframe for hotel_df

hotel_df = pd.DataFrame(weather_df, columns =['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name'] = ''

hotel_df.head(5)



In [252]:
# Set parameters to search for hotels with 5000 meters

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


# Loop through the hotel_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():
   
    lat = row['Lat']
    lng = row['Lng']
    

    # update address key value
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
        
    # make request
    
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (IndexError):
        print("Missing field/result... skipping.")    
            
      

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [272]:
# Visualize to confirm airport data appears

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Esmeraldas,BR,-19.7625,-44.3139,Hotel Fasano Belo Horizonte
1,Lyubinskiy,RU,55.1542,72.6925,ibis Sibir Omsk
2,Jamestown,US,42.0970,-79.2353,Chautauqua Harbor Hotel
3,Bengkulu,ID,-3.8004,102.2655,Mercure Bengkulu
4,Ushuaia,AR,-54.8000,-68.3000,Los Cauquenes Resort & Spa
...,...,...,...,...,...
584,Marabu,MY,4.4148,114.0089,Miri Marriott Resort & Spa
585,Margate,GB,51.3813,1.3862,Botany Bay Hotel
586,Calabozo,VE,8.9242,-67.4293,Best Western Hotel Plaza Real
587,Leh,IN,34.1667,77.5833,The Grand Dragon Ladakh


In [263]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [275]:
# Add marker layer ontop of heat map
# Create a map using state centroid coordinates to set markers

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


# Create a marker_layer using the poverty list to fill the info box

fig.add_layer(markers)  


# Display figure
fig

# Save the image as .pngs

plt.savefig('images/hotel_map.png')


<Figure size 432x288 with 0 Axes>